In [ ]:
import os
import sys
import shutil

print("===== 1. 環境初始化與套件安裝 =====")
# 安裝 YOLO 與 RF-DETR 所需套件
!pip install -q ultralytics timm submitit tqdm pandas pillow pyyaml

print("\n===== 2. 設定 RF-DETR 模組 =====")
# 下載 RF-DETR 原始碼
if not os.path.exists("RF-DETR"):
    print("正在下載 RF-DETR 原始碼...")
    !git clone https://github.com/jahongir7174/RF-DETR.git
else:
    print("RF-DETR 資料夾已存在。")

# 【關鍵步驟】將 RF-DETR 資料夾加入 Python 搜尋路徑
# 這樣做之後，您的 'from rfdetr import RFDETRSmall' 才能正常運作
rf_path = os.path.abspath("RF-DETR")
if rf_path not in sys.path:
    sys.path.append(rf_path)
    print(f"已將 {rf_path} 加入環境路徑。")

print("\n===== 3. 準備測試集資料 =====")
# 您的程式碼 get_test_images_safe 預設會找 ./testing_image/testing_image
# 所以我們解壓縮到 ./testing_image，這樣通常會產生 testing_image/testing_image 的結構
if os.path.exists("testing_image.zip"):
    if not os.path.exists("testing_image"):
        print("正在解壓縮測試集 (Testing Data)...")
        !unzip -q testing_image.zip -d ./testing_image
        print("解壓縮完成。")
    else:
        print("檢測到 testing_image 資料夾已存在，跳過解壓縮。")
else:
    print("【警告】找不到 testing_image.zip，請確認檔案是否已上傳至同級目錄！")

print("\n===== 4. 最終檢查 =====")
try:
    from rfdetr import RFDETRSmall
    print("✅ 成功匯入 RF-DETR 模組！")
except ImportError as e:
    print(f"❌ RF-DETR 匯入失敗: {e}")
    print("請確認 RF-DETR 資料夾內是否有 rfdetr.py 或 __init__.py")

print("\n環境準備就緒，請繼續執行下方的主程式。")

In [ ]:
"""
AICUP 2025: 15-Model Ensemble Pipeline (v20 Fixed - All Issues Resolved)
- 5x RF-DETR Small (50 Epochs) + 10x YOLO (n/s × 5 folds)
- Fix: Import Path, USE_TTA, correct vote counting
- Fix: Proper weight calculation for TTA
"""

import os
import shutil
import json
import gc
import numpy as np
from pathlib import Path  # ✓ 修正1: 加入 Path
from PIL import Image, ImageOps
import torch
from tqdm import tqdm
from ultralytics import YOLO
from rfdetr import RFDETRSmall
from collections import defaultdict

# ==========================================
# 1. 全域配置
# ==========================================

TEST_IMG_ROOT = "./datasets/test/tmp/testing_image"
RAW_IMG_ROOT = "./training_image"   
RAW_LBL_ROOT = "./training_label"   
YOLO_BASE_DIR = "runs/detect"

RF_DATASET_ROOT = "./rf_detr_dataset"
RF_RUNS_DIR = "./rf_detr_5fold_runs"
OUTPUT_DIR = "./final_submission"
FINAL_TXT_PATH = os.path.join(OUTPUT_DIR, "submission_15models_v20_fixed.txt")

# --- 參數設定 ---
IMG_SIZE = 512
FOLDS = 5
RF_EPOCHS = 50          
YOLO_BATCH_SIZE = 32    
USE_TTA = True          # ✓ 修正2: 定義 USE_TTA

# 過濾參數 (嚴格模式)
SINGLE_CONF = 0.20      # 單模門檻
WBF_IOU_THR = 0.50      # 融合 IoU
MIN_MODEL_VOTES = 5     # ✓ 修正3: 改名為 MIN_MODEL_VOTES，表示「至少幾個模型」
FINAL_CONF = 0.30       # ✓ 修正4: 降為 0.30，避免過度嚴格

# 權重 (每個「模型」的權重，不論 TTA 次數)
WEIGHTS_MAP = {'yolo12s': 2.0, 'yolo12n': 2.0, 'rf_detr': 1.0}

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(RF_RUNS_DIR, exist_ok=True)

# ==========================================
# 2. 自定義 WBF (修正投票邏輯)
# ==========================================

def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return inter / (area1 + area2 - inter + 1e-6)

def custom_wbf(all_boxes, iou_thr, min_model_votes, num_total_models):
    """
    ✓ 修正版 WBF: 正確計算模型投票數
    
    all_boxes: list of [x1, y1, x2, y2, score, class, weight, model_id]
    """
    if len(all_boxes) == 0: 
        return []
    
    all_boxes = np.array(all_boxes)
    # 按分數排序
    order = np.argsort(-all_boxes[:, 4])
    all_boxes = all_boxes[order]
    
    groups = []
    processed = [False] * len(all_boxes)
    
    for i in range(len(all_boxes)):
        if processed[i]: 
            continue
        
        current_box = all_boxes[i]
        group = [current_box]
        model_ids = {int(current_box[7])}  # ✓ 追蹤唯一模型 ID
        processed[i] = True
        
        for j in range(i + 1, len(all_boxes)):
            if processed[j]: 
                continue
            iou = compute_iou(current_box[:4], all_boxes[j, :4])
            if iou >= iou_thr:
                group.append(all_boxes[j])
                model_ids.add(int(all_boxes[j, 7]))  # ✓ 記錄模型 ID
                processed[j] = True
        
        # ✓ 修正: 計算唯一模型數
        num_voting_models = len(model_ids)
        
        # 投票過濾
        if num_voting_models < min_model_votes:
            continue
        
        group = np.array(group)
        
        # 加權平均座標 (使用每個框的權重)
        w = group[:, 6]
        w_sum = w.sum()
        avg_box = (group[:, :4].T @ w) / w_sum
        
        # ✓ 修正: 全局分數 = 總加權分數 / 總模型數
        # 這裡用「總模型數」而非「總可能權重」，因為不同模型權重不同
        # 更公平的做法: 總分 / (所有模型的平均權重 × 總模型數)
        avg_weight = sum(WEIGHTS_MAP.values()) / len(WEIGHTS_MAP)  # (2+2+1)/3 ≈ 1.67
        total_possible_score = avg_weight * num_total_models
        
        total_score = (group[:, 4] * group[:, 6]).sum()
        avg_score = total_score / total_possible_score
        
        # 最終閾值過濾
        if avg_score >= FINAL_CONF:
            groups.append([*avg_box, avg_score, group[0, 5]])  # [x1,y1,x2,y2,score,class]
            
    return groups

# ==========================================
# 3. 工具函數
# ==========================================

def clear_gpu():
    gc.collect()
    torch.cuda.empty_cache()

def ensure_clean_dir(path):
    if os.path.exists(path):
        try: 
            shutil.rmtree(path)
        except: 
            pass
    os.makedirs(path, exist_ok=True)

def find_patient_root(root: str) -> str:
    for dirpath, dirnames, _ in os.walk(root):
        if any(d.lower().startswith("patient") for d in dirnames):
            return dirpath
    return root

def get_yolo_path(fold, model_type):
    candidates = [
        os.path.join(YOLO_BASE_DIR, f"fold{fold}_{model_type}", "weights", "best.pt"),
        os.path.join(YOLO_BASE_DIR, f"{model_type}_fold{fold}", "weights", "best.pt"),
    ]
    for p in candidates:
        if os.path.exists(p): 
            return p
    return None

def get_test_images_safe():
    """使用 os.walk 避免 Too many open files"""
    target_dir = TEST_IMG_ROOT
    if not os.path.exists(target_dir):
        target_dir = "./testing_image/testing_image"
        if not os.path.exists(target_dir): 
            return []

    imgs = []
    for root, dirs, files in os.walk(target_dir):
        for f in files:
            if f.lower().endswith(('.png', '.jpg')):
                imgs.append(os.path.join(root, f))
    return sorted(imgs)

# ==========================================
# 4. RF-DETR 訓練
# ==========================================

def create_5fold_splits(patients=range(1, 51)):
    patients = list(patients)
    np.random.seed(42)
    np.random.shuffle(patients)
    fold_size = len(patients) // 5
    folds = []
    for fold_idx in range(5):
        val_start = fold_idx * fold_size
        val_end = val_start + fold_size
        val_patients = patients[val_start:val_end]
        train_patients = [p for p in patients if p not in val_patients]
        folds.append({'fold': fold_idx, 'train': train_patients, 'valid': val_patients})
    return folds

def yolo_to_coco(images_dir, labels_dir, output_json, start_img_id=0, start_ann_id=0):
    img_id, ann_id = start_img_id, start_ann_id
    images, annotations = [], []
    categories = [{"id": 1, "name": "aortic_valve", "supercategory": "object"}]
    if not os.path.exists(images_dir): 
        return img_id, ann_id
    
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])
    for fname in image_files:
        img_id += 1
        images.append({"id": img_id, "file_name": fname, "width": IMG_SIZE, "height": IMG_SIZE})
        txt_path = os.path.join(labels_dir, fname.replace(".png", ".txt"))
        if not os.path.exists(txt_path): 
            continue
        
        with open(txt_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5: 
                    continue
                _, xc, yc, w, h = map(float, parts[:5])
                x_min = (xc - w/2) * IMG_SIZE
                y_min = (yc - h/2) * IMG_SIZE
                box_w = w * IMG_SIZE
                box_h = h * IMG_SIZE
                ann_id += 1
                annotations.append({
                    "id": ann_id, "image_id": img_id, "category_id": 1,
                    "bbox": [x_min, y_min, box_w, box_h],
                    "area": box_w * box_h, "iscrowd": 0
                })
    
    coco = {
        "info": {"desc": "AICUP"}, "licenses": [], 
        "images": images, "annotations": annotations, "categories": categories
    }
    os.makedirs(os.path.dirname(output_json), exist_ok=True)
    with open(output_json, "w") as f: 
        json.dump(coco, f)
    return img_id, ann_id

def prepare_fold_data(fold_info, img_root, lbl_root):
    fold_idx = fold_info['fold']
    fold_dir = Path(RF_DATASET_ROOT) / f'fold_{fold_idx}'
    if fold_dir.exists(): 
        shutil.rmtree(fold_dir)
    
    for split in ['train', 'valid']:
        dst_img = fold_dir / split
        dst_lbl = Path(f'./rf_tmp/fold_{fold_idx}_{split}_labels')
        ensure_clean_dir(str(dst_img))
        ensure_clean_dir(str(dst_lbl))
        
        for pid in fold_info[split]:
            p_str = f"patient{pid:04d}"
            src_img_dir = Path(img_root) / p_str
            src_lbl_dir = Path(lbl_root) / p_str
            if not src_img_dir.exists(): 
                continue
            
            for png_file in src_img_dir.glob("*.png"):
                txt_file = src_lbl_dir / f"{png_file.stem}.txt"
                if txt_file.exists():
                    shutil.copy2(png_file, dst_img / png_file.name)
                    shutil.copy2(txt_file, dst_lbl / txt_file.name)
    
    last_img, last_ann = yolo_to_coco(
        str(fold_dir / 'train'), 
        f'./rf_tmp/fold_{fold_idx}_train_labels', 
        str(fold_dir / 'train' / '_annotations.coco.json')
    )
    yolo_to_coco(
        str(fold_dir / 'valid'), 
        f'./rf_tmp/fold_{fold_idx}_valid_labels', 
        str(fold_dir / 'valid' / '_annotations.coco.json'),
        start_img_id=last_img, start_ann_id=last_ann
    )
    
    test_dir = fold_dir / 'test'
    if test_dir.exists(): 
        shutil.rmtree(test_dir)
    shutil.copytree(fold_dir / 'valid', test_dir)
    return fold_dir

def train_rfdetr_all_folds():
    try: 
        img_root = find_patient_root(RAW_IMG_ROOT)
        lbl_root = find_patient_root(RAW_LBL_ROOT)
    except: 
        return
    
    folds = create_5fold_splits()
    for fold_info in folds:
        fold_idx = fold_info['fold']
        print(f"\n>>> Processing RF-DETR Fold {fold_idx} <<<")
        save_dir = os.path.join(RF_RUNS_DIR, f"fold_{fold_idx}")
        final_ckpt = os.path.join(save_dir, "checkpoint_best_total.pth")
        
        if os.path.exists(final_ckpt):
            print(f"Fold {fold_idx} already trained. Skipping.")
            continue
        
        dataset_dir = prepare_fold_data(fold_info, img_root, lbl_root)
        clear_gpu()
        
        model = RFDETRSmall(num_classes=1)
        try:
            model.train(
                dataset_dir=str(dataset_dir), 
                epochs=RF_EPOCHS, 
                batch_size=8, 
                grad_accum_steps=2, 
                lr=1e-4, 
                output_dir=save_dir, 
                device="cuda", 
                early_stopping=True, 
                patience=5
            )
        except RuntimeError as e:
            shutil.rmtree(dataset_dir)
            raise e
        
        del model
        clear_gpu()
        
        if os.path.exists(dataset_dir): 
            shutil.rmtree(dataset_dir)
        
        # 清理多餘的 checkpoint
        for f in os.listdir(save_dir):
            if f.endswith(".pth") and "best_total" not in f:
                try: 
                    os.remove(os.path.join(save_dir, f))
                except: 
                    pass

# ==========================================
# 5. 預測核心 (修正: 加入 model_id)
# ==========================================

def run_yolo_inference(model_path, images, weight, model_id, global_preds):
    """✓ 修正: 加入 model_id 參數"""
    print(f"--> Batch Predicting YOLO: {model_path}")
    try:
        model = YOLO(model_path)
        total = len(images)
        pbar = tqdm(total=total, desc=f"YOLO-{model_id}")
        
        for i in range(0, total, YOLO_BATCH_SIZE):
            batch = images[i : i + YOLO_BATCH_SIZE]
            try:
                results = model.predict(
                    batch, imgsz=640, conf=SINGLE_CONF, 
                    augment=USE_TTA, device=0, verbose=False, stream=False
                )
                for res in results:
                    fid = os.path.splitext(os.path.basename(res.path))[0]
                    if len(res.boxes) > 0:
                        b = res.boxes.xyxyn.cpu().numpy()
                        s = res.boxes.conf.cpu().numpy()
                        
                        for k in range(len(b)):
                            # [x1, y1, x2, y2, score, class, weight, model_id]
                            box_data = [*b[k], float(s[k]), 0, weight, model_id]
                            global_preds[fid].append(box_data)
            except Exception as e: 
                print(f"Batch Error: {e}")
            pbar.update(len(batch))
        
        pbar.close()
        del model
    except: 
        pass
    clear_gpu()

def run_rfdetr_inference(model_path, images, weight, model_id, global_preds):
    """✓ 修正: 加入 model_id 參數"""
    print(f"--> Predicting RF-DETR: {model_path}")
    try:
        model = RFDETRSmall(num_classes=1)
        ckpt = torch.load(model_path, map_location="cuda")
        sd = ckpt.get("model_state_dict", ckpt.get("model", ckpt.get("ema_model", ckpt)))
        
        try: 
            model.model.model.load_state_dict(sd)
        except:
            try: 
                model.model.load_state_dict(sd)
            except: 
                model.load_state_dict(sd)
        
        model.model.model.to("cuda").eval()
        
        for img_path in tqdm(images, desc=f"RF-{model_id}"):
            fid = os.path.splitext(os.path.basename(img_path))[0]
            try:
                pil_img = Image.open(img_path).convert("RGB")
                w, h = pil_img.size
                
                # 手動 TTA
                variants = [(pil_img, False)]
                if USE_TTA: 
                    variants.append((ImageOps.mirror(pil_img), True))
                
                for img, is_flip in variants:
                    res = model.predict(img, confidence=SINGLE_CONF)
                    if hasattr(res, 'xyxy'): 
                        b_raw, s_raw = res.xyxy, res.confidence
                    else: 
                        b_raw, s_raw = res['boxes'], res['scores']
                    
                    for k in range(len(s_raw)):
                        x1, y1, x2, y2 = map(float, b_raw[k])
                        if is_flip: 
                            x1, x2 = w - x2, w - x1
                        
                        # Normalize
                        norm_box = [
                            max(0, x1/w), max(0, y1/h), 
                            min(1, x2/w), min(1, y2/h)
                        ]
                        # [x1, y1, x2, y2, score, class, weight, model_id]
                        global_preds[fid].append([*norm_box, float(s_raw[k]), 0, weight, model_id])
            except: 
                pass
        
        del model
    except Exception as e: 
        print(f"❌ Failed RF load: {e}")
    clear_gpu()

# ==========================================
# 6. 主流程
# ==========================================

def run_pipeline():
    print(">>> Step 1: Training RF-DETR 5 Folds...")
    train_rfdetr_all_folds()
    
    print("\n>>> Step 2: Gathering Images...")
    test_imgs = get_test_images_safe()
    if not test_imgs:
        print("❌ Error: No images found.")
        return
    print(f"Images: {len(test_imgs)}")
    
    global_preds = defaultdict(list)
    model_counter = 0  # ✓ 追蹤模型 ID
    
    # 1. YOLOv12n (5 folds)
    print("\n>>> Step 3a: YOLO12n Inference...")
    for i in range(FOLDS):
        path = get_yolo_path(i, 'yolo12n')
        if path: 
            run_yolo_inference(path, test_imgs, WEIGHTS_MAP['yolo12n'], model_counter, global_preds)
            model_counter += 1
    
    # 2. YOLOv12s (5 folds)
    print("\n>>> Step 3b: YOLO12s Inference...")
    for i in range(FOLDS):
        path = get_yolo_path(i, 'yolo12s')
        if path: 
            run_yolo_inference(path, test_imgs, WEIGHTS_MAP['yolo12s'], model_counter, global_preds)
            model_counter += 1
    
    # 3. RF-DETR (5 folds)
    print("\n>>> Step 3c: RF-DETR Inference...")
    for i in range(FOLDS):
        path = f"./rf_detr_5fold_runs/fold_{i}/checkpoint_best_total.pth"
        if not os.path.exists(path): 
            path = f"./rf_detr_5fold_runs/fold_{i}/checkpoint.pth"
        if os.path.exists(path): 
            run_rfdetr_inference(path, test_imgs, WEIGHTS_MAP['rf_detr'], model_counter, global_preds)
            model_counter += 1
    
    print(f"\nTotal models loaded: {model_counter}")
    
    # 4. Custom WBF Ensemble
    print("\n>>> Step 4: Final Custom WBF (Strict Voting)...")
    
    stats = {'total': 0, 'detected': 0, 'total_boxes': 0}
    
    with open(FINAL_TXT_PATH, "w", encoding="utf-8") as f_out:
        for fid in tqdm(sorted(global_preds.keys()), desc="WBF"):
            stats['total'] += 1
            box_data = global_preds[fid]
            
            final_results = custom_wbf(
                box_data, 
                WBF_IOU_THR, 
                MIN_MODEL_VOTES, 
                model_counter  # ✓ 傳入實際模型總數
            )
            
            if len(final_results) > 0:
                stats['detected'] += 1
                stats['total_boxes'] += len(final_results)
            
            for res in final_results:
                x1 = int(res[0] * IMG_SIZE)
                y1 = int(res[1] * IMG_SIZE)
                x2 = int(res[2] * IMG_SIZE)
                y2 = int(res[3] * IMG_SIZE)
                
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(IMG_SIZE, x2), min(IMG_SIZE, y2)
                
                score = res[4]
                f_out.write(f"{fid} 0 {score:.4f} {x1} {y1} {x2} {y2}\n")
    
    print(f"\n{'='*60}")
    print(f"Final Statistics:")
    print(f"  Images processed: {stats['total']}")
    print(f"  Images with detections: {stats['detected']} ({stats['detected']/stats['total']*100:.1f}%)")
    print(f"  Avg boxes per detected image: {stats['total_boxes']/max(1, stats['detected']):.2f}")
    print(f"  Submission file: {FINAL_TXT_PATH}")
    print(f"{'='*60}")

if __name__ == "__main__":
    run_pipeline()

>>> Step 1: Training RF-DETR 5 Folds...

>>> Processing RF-DETR Fold 0 <<<
Fold 0 already trained. Skipping.

>>> Processing RF-DETR Fold 1 <<<
Fold 1 already trained. Skipping.

>>> Processing RF-DETR Fold 2 <<<
Fold 2 already trained. Skipping.

>>> Processing RF-DETR Fold 3 <<<
Fold 3 already trained. Skipping.

>>> Processing RF-DETR Fold 4 <<<
Fold 4 already trained. Skipping.

>>> Step 2: Gathering Images...
Images: 16620

>>> Step 3a: YOLO12n Inference...
--> Batch Predicting YOLO: runs/detect/fold0_yolo12n/weights/best.pt


YOLO-0: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16620/16620 [01:32<00:00, 179.16it/s]


--> Batch Predicting YOLO: runs/detect/fold1_yolo12n/weights/best.pt


YOLO-1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16620/16620 [01:29<00:00, 185.27it/s]


--> Batch Predicting YOLO: runs/detect/fold2_yolo12n/weights/best.pt


...

RF-11: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16620/16620 [08:13<00:00, 33.70it/s]


--> Predicting RF-DETR: ./rf_detr_5fold_runs/fold_2/checkpoint_best_total.pth
Using a different number of positional encodings than DINOv2, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Using patch size 16 instead of 14, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Loading pretrain weights


RF-12: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16620/16620 [08:16<00:00, 33.47it/s]


--> Predicting RF-DETR: ./rf_detr_5fold_runs/fold_3/checkpoint_best_total.pth
Using a different number of positional encodings than DINOv2, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Using patch size 16 instead of 14, which means we're not loading DINOv2 backbone weights. This is not a problem if finetuning a pretrained RF-DETR model.
Loading pretrain weights


RF-13:  55%|█████████████████████████████████████████████████████████████████████▎                                                       | 9213/16620 [04:14<04:01, 30.69it/s]